## CBoW ਮਾਡਲ ਦੀ ਟ੍ਰੇਨਿੰਗ

ਇਹ ਨੋਟਬੁੱਕ [AI for Beginners Curriculum](http://aka.ms/ai-beginners) ਦਾ ਹਿੱਸਾ ਹੈ।

ਇਸ ਉਦਾਹਰਨ ਵਿੱਚ, ਅਸੀਂ ਆਪਣਾ Word2Vec ਐਮਬੈਡਿੰਗ ਸਪੇਸ ਪ੍ਰਾਪਤ ਕਰਨ ਲਈ CBoW ਭਾਸ਼ਾ ਮਾਡਲ ਦੀ ਟ੍ਰੇਨਿੰਗ ਦੇਖਾਂਗੇ। ਅਸੀਂ ਟੈਕਸਟ ਦੇ ਸਰੋਤ ਵਜੋਂ AG News ਡੇਟਾਸੈਟ ਦੀ ਵਰਤੋਂ ਕਰਾਂਗੇ।


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ਪਹਿਲਾਂ ਆਓ ਅਸੀਂ ਆਪਣਾ ਡਾਟਾਸੈੱਟ ਲੋਡ ਕਰੀਏ ਅਤੇ ਟੋਕਨਾਈਜ਼ਰ ਅਤੇ ਸ਼ਬਦਾਵਲੀ ਨੂੰ ਪਰਿਭਾਸ਼ਿਤ ਕਰੀਏ। ਅਸੀਂ ਗਣਨਾਵਾਂ ਨੂੰ ਥੋੜ੍ਹਾ ਸੀਮਿਤ ਕਰਨ ਲਈ `vocab_size` ਨੂੰ 5000 'ਤੇ ਸੈਟ ਕਰਾਂਗੇ।


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoW ਮਾਡਲ

CBoW $2N$ ਨੇੜਲੇ ਸ਼ਬਦਾਂ ਦੇ ਆਧਾਰ 'ਤੇ ਇੱਕ ਸ਼ਬਦ ਦੀ ਭਵਿੱਖਵਾਣੀ ਕਰਨਾ ਸਿੱਖਦਾ ਹੈ। ਉਦਾਹਰਣ ਵਜੋਂ, ਜਦੋਂ $N=1$, ਸਾਨੂੰ ਵਾਕ *I like to train networks* ਤੋਂ ਹੇਠਲੇ ਜੋੜ ਮਿਲਣਗੇ: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks)। ਇੱਥੇ, ਪਹਿਲਾ ਸ਼ਬਦ ਨੇੜਲੇ ਸ਼ਬਦ ਵਜੋਂ ਇਨਪੁਟ ਲਈ ਵਰਤਿਆ ਜਾਂਦਾ ਹੈ, ਅਤੇ ਦੂਜਾ ਸ਼ਬਦ ਉਹ ਹੈ ਜਿਸਦੀ ਭਵਿੱਖਵਾਣੀ ਕੀਤੀ ਜਾ ਰਹੀ ਹੈ।

ਅਗਲਾ ਸ਼ਬਦ ਭਵਿੱਖਵਾਣੀ ਕਰਨ ਲਈ ਨੈਟਵਰਕ ਬਣਾਉਣ ਲਈ, ਸਾਨੂੰ ਨੇੜਲੇ ਸ਼ਬਦ ਨੂੰ ਇਨਪੁਟ ਵਜੋਂ ਪ੍ਰਦਾਨ ਕਰਨਾ ਪਵੇਗਾ, ਅਤੇ ਸ਼ਬਦ ਨੰਬਰ ਨੂੰ ਆਉਟਪੁੱਟ ਵਜੋਂ ਪ੍ਰਾਪਤ ਕਰਨਾ ਪਵੇਗਾ। CBoW ਨੈਟਵਰਕ ਦੀ ਆਰਕੀਟੈਕਚਰ ਹੇਠਾਂ ਦਿੱਤੀ ਗਈ ਹੈ:

* ਇਨਪੁਟ ਸ਼ਬਦ ਨੂੰ ਐਮਬੈਡਿੰਗ ਲੇਅਰ ਰਾਹੀਂ ਪਾਸ ਕੀਤਾ ਜਾਂਦਾ ਹੈ। ਇਹ ਐਮਬੈਡਿੰਗ ਲੇਅਰ ਸਾਡਾ Word2Vec ਐਮਬੈਡਿੰਗ ਹੋਵੇਗਾ, ਇਸ ਲਈ ਅਸੀਂ ਇਸਨੂੰ `embedder` ਵੈਰੀਏਬਲ ਵਜੋਂ ਅਲੱਗ ਤੌਰ 'ਤੇ ਪਰਿਭਾਸ਼ਿਤ ਕਰਾਂਗੇ। ਇਸ ਉਦਾਹਰਣ ਵਿੱਚ ਅਸੀਂ ਐਮਬੈਡਿੰਗ ਸਾਈਜ਼ = 30 ਵਰਤਾਂਗੇ, ਹਾਲਾਂਕਿ ਤੁਸੀਂ ਉੱਚੇ ਡਾਈਮੈਂਸ਼ਨ ਨਾਲ ਅਨੁਭਵ ਕਰਨਾ ਚਾਹੁੰਦੇ ਹੋ (ਅਸਲ Word2Vec ਵਿੱਚ 300 ਹੈ)
* ਐਮਬੈਡਿੰਗ ਵੈਕਟਰ ਨੂੰ ਫਿਰ ਇੱਕ ਲੀਨੀਅਰ ਲੇਅਰ ਵਿੱਚ ਪਾਸ ਕੀਤਾ ਜਾਵੇਗਾ ਜੋ ਆਉਟਪੁੱਟ ਸ਼ਬਦ ਦੀ ਭਵਿੱਖਵਾਣੀ ਕਰੇਗਾ। ਇਸ ਲਈ ਇਸ ਵਿੱਚ `vocab_size` ਨਿਊਰੋਨ ਹਨ।

ਆਉਟਪੁੱਟ ਲਈ, ਜੇਕਰ ਅਸੀਂ `CrossEntropyLoss` ਨੂੰ ਲਾਸ ਫੰਕਸ਼ਨ ਵਜੋਂ ਵਰਤਦੇ ਹਾਂ, ਤਾਂ ਸਾਨੂੰ ਸਿਰਫ਼ ਸ਼ਬਦ ਨੰਬਰਾਂ ਨੂੰ ਉਮੀਦਵਾਰ ਨਤੀਜੇ ਵਜੋਂ ਪ੍ਰਦਾਨ ਕਰਨਾ ਪਵੇਗਾ, ਬਿਨਾਂ ਇੱਕ-ਹਾਟ ਐਨਕੋਡਿੰਗ।


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## ਟ੍ਰੇਨਿੰਗ ਡਾਟਾ ਤਿਆਰ ਕਰਨਾ

ਹੁਣ ਆਓ ਮੁੱਖ ਫੰਕਸ਼ਨ ਪ੍ਰੋਗਰਾਮ ਕਰੀਏ ਜੋ ਟੈਕਸਟ ਤੋਂ CBoW ਸ਼ਬਦ ਜੋੜੇ ਦੀ ਗਣਨਾ ਕਰੇਗਾ। ਇਹ ਫੰਕਸ਼ਨ ਸਾਨੂੰ ਵਿੰਡੋ ਸਾਈਜ਼ ਨਿਰਧਾਰਤ ਕਰਨ ਦੀ ਆਗਿਆ ਦੇਵੇਗਾ ਅਤੇ ਸ਼ਬਦਾਂ ਦੇ ਜੋੜੇ - ਇਨਪੁਟ ਅਤੇ ਆਉਟਪੁਟ ਸ਼ਬਦ ਵਾਪਸ ਕਰੇਗਾ। ਧਿਆਨ ਦਿਓ ਕਿ ਇਹ ਫੰਕਸ਼ਨ ਸ਼ਬਦਾਂ 'ਤੇ ਵਰਤਿਆ ਜਾ ਸਕਦਾ ਹੈ, ਜਿਵੇਂ ਕਿ ਵੈਕਟਰ/ਟੈਂਸਰ 'ਤੇ - ਜੋ ਸਾਨੂੰ ਟੈਕਸਟ ਨੂੰ ਐਨਕੋਡ ਕਰਨ ਦੀ ਆਗਿਆ ਦੇਵੇਗਾ, ਇਸਨੂੰ `to_cbow` ਫੰਕਸ਼ਨ ਨੂੰ ਪਾਸ ਕਰਨ ਤੋਂ ਪਹਿਲਾਂ।


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


ਚਲੋ ਪ੍ਰਸ਼ਿਕਸ਼ਣ ਡੇਟਾਸੈੱਟ ਤਿਆਰ ਕਰੀਏ। ਅਸੀਂ ਸਾਰੀ ਖ਼ਬਰਾਂ ਵਿੱਚੋਂ ਗੁਜ਼ਰਾਂਗੇ, `to_cbow` ਨੂੰ ਕਾਲ ਕਰਾਂਗੇ ਤਾਂ ਜੋ ਸ਼ਬਦ ਜੋੜਿਆਂ ਦੀ ਸੂਚੀ ਮਿਲ ਸਕੇ, ਅਤੇ ਉਹਨਾਂ ਜੋੜਿਆਂ ਨੂੰ `X` ਅਤੇ `Y` ਵਿੱਚ ਸ਼ਾਮਲ ਕਰਾਂਗੇ। ਸਮੇਂ ਦੀ ਬਚਤ ਲਈ, ਅਸੀਂ ਸਿਰਫ ਪਹਿਲੀਆਂ 10 ਹਜ਼ਾਰ ਖ਼ਬਰਾਂ ਨੂੰ ਹੀ ਧਿਆਨ ਵਿੱਚ ਲਵਾਂਗੇ - ਜੇ ਤੁਹਾਡੇ ਕੋਲ ਹੋਰ ਸਮਾਂ ਹੈ ਅਤੇ ਤੁਸੀਂ ਵਧੀਆ ਐਮਬੈਡਿੰਗ ਪ੍ਰਾਪਤ ਕਰਨਾ ਚਾਹੁੰਦੇ ਹੋ, ਤਾਂ ਤੁਸੀਂ ਇਸ ਸੀਮਾ ਨੂੰ ਆਸਾਨੀ ਨਾਲ ਹਟਾ ਸਕਦੇ ਹੋ :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

ਅਸੀਂ ਉਸ ਡਾਟਾ ਨੂੰ ਇੱਕ ਡਾਟਾਸੈਟ ਵਿੱਚ ਰੂਪਾਂਤਰਿਤ ਕਰਾਂਗੇ, ਅਤੇ ਡਾਟਾਲੋਡਰ ਬਣਾਵਾਂਗੇ:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

ਅਸੀਂ ਉਸ ਡਾਟਾ ਨੂੰ ਇੱਕ ਡਾਟਾਸੈਟ ਵਿੱਚ ਰੂਪਾਂਤਰਿਤ ਕਰਾਂਗੇ, ਅਤੇ ਡਾਟਾਲੋਡਰ ਬਣਾਵਾਂਗੇ:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

ਹੁਣ ਆਓ ਅਸਲ ਟ੍ਰੇਨਿੰਗ ਕਰੀਏ। ਅਸੀਂ `SGD` ਓਪਟੀਮਾਈਜ਼ਰ ਨੂੰ ਕਾਫ਼ੀ ਉੱਚੀ ਲਰਨਿੰਗ ਰੇਟ ਨਾਲ ਵਰਤਾਂਗੇ। ਤੁਸੀਂ ਹੋਰ ਓਪਟੀਮਾਈਜ਼ਰਾਂ, ਜਿਵੇਂ ਕਿ `Adam`, ਨਾਲ ਵੀ ਅਜ਼ਮਾਉਣ ਦੀ ਕੋਸ਼ਿਸ਼ ਕਰ ਸਕਦੇ ਹੋ। ਅਸੀਂ ਸ਼ੁਰੂਆਤ ਵਿੱਚ 10 ਈਪੋਕ ਲਈ ਟ੍ਰੇਨ ਕਰਾਂਗੇ - ਅਤੇ ਜੇ ਤੁਸੀਂ ਹੋਰ ਘੱਟ ਲਾਸ਼ ਚਾਹੁੰਦੇ ਹੋ ਤਾਂ ਇਸ ਸੈੱਲ ਨੂੰ ਦੁਬਾਰਾ ਚਲਾ ਸਕਦੇ ਹੋ।


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## ਵਰਡ2ਵੈਕ ਨੂੰ ਅਜ਼ਮਾਉਣਾ

ਵਰਡ2ਵੈਕ ਵਰਤਣ ਲਈ, ਆਓ ਆਪਣੇ ਸ਼ਬਦ-ਭੰਡਾਰ ਵਿੱਚ ਮੌਜੂਦ ਸਾਰੇ ਸ਼ਬਦਾਂ ਲਈ ਵੈਕਟਰ ਕੱਢੀਏ:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

ਚਲੋ ਵੇਖੀਏ, ਉਦਾਹਰਨ ਵਜੋਂ, ਸ਼ਬਦ **Paris** ਨੂੰ ਇੱਕ ਵੇਕਟਰ ਵਿੱਚ ਕਿਵੇਂ ਕੋਡ ਕੀਤਾ ਜਾਂਦਾ ਹੈ:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


ਇਹ Word2Vec ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਪਰਯਾਏਵਾਚੀ ਸ਼ਬਦ ਲੱਭਣਾ ਦਿਲਚਸਪ ਹੈ। ਹੇਠਾਂ ਦਿੱਤੀ ਫੰਕਸ਼ਨ ਦਿੱਤੇ ਗਏ ਇਨਪੁਟ ਲਈ `n` ਸਭ ਤੋਂ ਨੇੜਲੇ ਸ਼ਬਦ ਵਾਪਸ ਕਰੇਗੀ। ਉਨ੍ਹਾਂ ਨੂੰ ਲੱਭਣ ਲਈ, ਅਸੀਂ $|w_i - v|$ ਦਾ ਨਾਰਮ ਗਣਨਾ ਕਰਦੇ ਹਾਂ, ਜਿੱਥੇ $v$ ਸਾਡੇ ਇਨਪੁਟ ਸ਼ਬਦ ਨਾਲ ਸਬੰਧਤ ਵੇਕਟਰ ਹੈ, ਅਤੇ $w_i$ ਸ਼ਬਦਾਵਲੀ ਵਿੱਚ $i$ਵੇਂ ਸ਼ਬਦ ਦਾ ਐਨਕੋਡਿੰਗ ਹੈ। ਫਿਰ ਅਸੀਂ ਐਰੇ ਨੂੰ ਸੌਰਟ ਕਰਦੇ ਹਾਂ ਅਤੇ `argsort` ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਸੰਬੰਧਤ ਇੰਡੈਕਸ ਵਾਪਸ ਕਰਦੇ ਹਾਂ, ਅਤੇ ਸੂਚੀ ਦੇ ਪਹਿਲੇ `n` ਤੱਤ ਲੈਂਦੇ ਹਾਂ, ਜੋ ਸ਼ਬਦਾਵਲੀ ਵਿੱਚ ਸਭ ਤੋਂ ਨੇੜਲੇ ਸ਼ਬਦਾਂ ਦੇ ਸਥਾਨਾਂ ਨੂੰ ਐਨਕੋਡ ਕਰਦੇ ਹਨ।


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## ਮੁੱਖ ਗੱਲ

ਚਲਾਕ ਤਕਨੀਕਾਂ ਜਿਵੇਂ ਕਿ CBoW ਦੀ ਵਰਤੋਂ ਕਰਕੇ, ਅਸੀਂ Word2Vec ਮਾਡਲ ਨੂੰ ਟ੍ਰੇਨ ਕਰ ਸਕਦੇ ਹਾਂ। ਤੁਸੀਂ ਸਕਿਪ-ਗ੍ਰਾਮ ਮਾਡਲ ਨੂੰ ਟ੍ਰੇਨ ਕਰਨ ਦੀ ਕੋਸ਼ਿਸ਼ ਵੀ ਕਰ ਸਕਦੇ ਹੋ, ਜੋ ਕੇਂਦਰੀ ਸ਼ਬਦ ਦੇ ਆਧਾਰ 'ਤੇ ਨੇੜਲੇ ਸ਼ਬਦ ਦੀ ਭਵਿੱਖਵਾਣੀ ਕਰਨ ਲਈ ਟ੍ਰੇਨ ਕੀਤਾ ਜਾਂਦਾ ਹੈ, ਅਤੇ ਦੇਖ ਸਕਦੇ ਹੋ ਕਿ ਇਹ ਕਿੰਨਾ ਵਧੀਆ ਕੰਮ ਕਰਦਾ ਹੈ।



---

**ਅਸਵੀਕਰਤੀ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਜਦੋਂ ਕਿ ਅਸੀਂ ਸਹੀ ਹੋਣ ਦਾ ਯਤਨ ਕਰਦੇ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਦਿਓ ਕਿ ਸਵੈਚਾਲਿਤ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸੁੱਚਤਤਾਵਾਂ ਹੋ ਸਕਦੀਆਂ ਹਨ। ਇਸ ਦੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਮੌਜੂਦ ਮੂਲ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਪ੍ਰਮਾਣਿਕ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਇਸ ਅਨੁਵਾਦ ਦੇ ਪ੍ਰਯੋਗ ਤੋਂ ਪੈਦਾ ਹੋਣ ਵਾਲੇ ਕਿਸੇ ਵੀ ਗਲਤਫਹਿਮੀ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆ ਲਈ ਅਸੀਂ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।  
